In [1]:
import pandas as pd
import numpy as np
import math
from datetime import date, timedelta
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import warnings
warnings.simplefilter('ignore', np.RankWarning)

In [2]:
train = pd.read_csv("train.csv", parse_dates=["date"])

In [3]:
test = pd.read_csv("test.csv", parse_dates=["date"])

In [4]:
train.rename(columns={"date":"datetime"}, inplace=True)

In [5]:
all_date = [pd.to_datetime("2018-01-01 00:00:00")]

while(all_date[-1]!=pd.to_datetime("2018-12-31 23:00:00")):
    all_date.append(all_date[-1]+timedelta(hours=1))
    
train_2018 = pd.DataFrame({"datetime":all_date})

train_2018 = train_2018.merge(train[train.datetime.dt.year==2018], on="datetime", how="left").sort_values("datetime")

train_2018 = train_2018[train.columns]
# train_2018 = train[train.datetime.dt.year==2017].copy()
# train_2018.datetime = train_2018.datetime.apply(lambda x: x.replace(year=2018))
# train_2018 = train[train.datetime.dt.year==2018].merge(train_2018[["datetime"]], on="datetime", how="left")


# all_date = [pd.to_datetime("2018-01-01 00:00:00")]

# while(all_date[-1]!=pd.to_datetime("2018-12-31 23:00:00")):
#     all_date.append(all_date[-1]+timedelta(hours=1))
    
# train_2018_tmp = pd.DataFrame({"datetime":all_date})

# train_2018 = train_2018_tmp.merge(train_2018, on="datetime", how="left").sort_values("datetime")

# train_2018 = train_2018[train.columns]



train = pd.concat([train[train.datetime.dt.year==2017], train_2018])

In [6]:
train["date"] = train.datetime.dt.strftime("%Y-%m-%d")
# train["date_f"] = (train.datetime.dt.year - 2017)*365.25 + train.datetime.dt.month*365.25/12 + train.datetime.dt.day

train["time"] = train.datetime.dt.strftime("%H:%M:%S")
train["time_f"] = (train.datetime.dt.hour*60*60 + train.datetime.dt.minute*60 + train.datetime.dt.second)/(24*60*60)

# train["datetime_f"] = train.date_f + train.time_f

train["dweek"] = train.datetime.dt.dayofweek
train["dweek_time"] = train.dweek + train.time_f

train["week"] = train.datetime.dt.weekofyear

# Split

In [7]:
train_2017 = train[train.datetime.dt.year==2017].copy()
train_2018 = train[train.datetime.dt.year==2018].copy()

# Lookup Table adjusted by day min-max then closest template difference

In [8]:
train_train = train_2018.copy()

In [9]:
template2 = train_train.groupby(["dweek","time"]).speed.mean()

In [10]:
template_day_mean = train_train.groupby("dweek").speed.mean()

In [11]:
template_day_min_max = train_train.groupby("dweek").speed.std()

In [12]:
template_ref_src = template2.reset_index()
template_ref_src["prev"] = template_ref_src.speed.shift(1).fillna(float("inf"))
template_ref_src["next"] = template_ref_src.speed.shift(-1).fillna(float("inf"))

template_ref_src["ref"] = template_ref_src.apply(lambda x: "prev" if abs(x.prev-x.speed) < abs(x.next-x.speed) else "next", axis=1)
template_ref_src["ignoreOther"] = template_ref_src.apply(lambda x: (abs(x["next" if x.ref=="prev" else "prev"]-x.speed)/abs(x[x.ref]-x.speed) > 4), axis=1)

template_ref = template_ref_src.set_index(["dweek","time"]).ref
template_ignore = template_ref_src.set_index(["dweek","time"]).ignoreOther

In [13]:
def lookup(x, inputDF=train_2018):
    day_min_max = inputDF[(inputDF.week==x.week)&(inputDF.dweek==x.dweek)&(inputDF.time!=x.time)].speed.std()
    day_mean = inputDF[(inputDF.week==x.week)&(inputDF.dweek==x.dweek)&(inputDF.time!=x.time)].speed.mean()
    return ((template2.loc[x.dweek, x.time] - template_day_mean.loc[x.dweek])*day_min_max/template_day_min_max.loc[x.dweek]) + day_mean

In [14]:
tmp = train_2018.copy()

tmp["computed"] = tmp.apply(lambda x: lookup(x), axis=1)

In [15]:
tmp["diff"] = tmp.speed - tmp.computed

tmp["prev"] = tmp["diff"].fillna(method="ffill").shift(1)
tmp["next"] = tmp["diff"].fillna(method="bfill").shift(-1)

In [16]:
def lookup2(x):
    return x.computed + (((0 if np.isnan(x.prev) else x.prev) + (0 if np.isnan(x.next) else x.next))/2)

In [17]:
error = {}

for i in range(7):
    print(i)
    error[i] = {}
    result = tmp.copy()
#     result["computed3"] = result.apply(lookup2, axis=1)
    
    for j in range(0,30):#np.arange(1.99,3,0.01):
        if(j==0):
            result["computed3"] = result.apply(lookup2, axis=1)
            result["computed3_l"] = result["computed3"].copy()
            result["computed3_r"] = result["computed3"].copy()
        else:
            result["computed3"] = result.computed3.rolling(window=3, center=True).mean().fillna(result.computed3)
            result["computed3_l"] = result.computed3_l.rolling(window=3).mean().fillna(result.computed3_l)
            result.sort_values("datetime", ascending=False, inplace=True)
            result["computed3_r"] = result.computed3_r.rolling(window=3).mean().fillna(result.computed3_r)
            result.sort_values("datetime", ascending=True, inplace=True)
            

    #     result["computed3"] = result.computed2.rolling(window=3, win_type="gaussian", center=True).mean(std=j).fillna(result.computed2)
        for k in tmp.time.unique():
            mse_c = mean_squared_error(result[(result.dweek==i)&(result.time==k)&(result.speed.notna())].sort_values("id").speed, result[(result.dweek==i)&(result.time==k)&(result.speed.notna())].sort_values("id").computed3)
            mse_l = mean_squared_error(result[(result.dweek==i)&(result.time==k)&(result.speed.notna())].sort_values("id").speed, result[(result.dweek==i)&(result.time==k)&(result.speed.notna())].sort_values("id").computed3_l)
            mse_r = mean_squared_error(result[(result.dweek==i)&(result.time==k)&(result.speed.notna())].sort_values("id").speed, result[(result.dweek==i)&(result.time==k)&(result.speed.notna())].sort_values("id").computed3_r)
            
            if((mse_c<=mse_l)and(mse_c<=mse_r)):
                mse = mse_c
                direction = "c"
            elif((mse_l<=mse_c)and(mse_l<=mse_r)):
                mse = mse_l
                direction = "l"
            elif((mse_r<=mse_c)and(mse_r<=mse_l)):
                mse = mse_r
                direction = "r"
            else:
                mse = None
                direction = None
            
            if(k not in error[i].keys()):
                error[i][k] = (round(j,2), mse, direction, None)
            elif(mse<error[i][k][1]):
                error[i][k] = (round(j,2), mse, direction, False)
            elif((mse>error[i][k][1]) and (error[i][k][3]==False)):
                error[i][k] = (error[i][k][0], error[i][k][1], error[i][k][2], True)

0
1
2
3
4
5
6


In [18]:
optimal_smooth = error

In [19]:
smooth_cnt = list(pd.Series([optimal_smooth[i][j][0] for j in tmp.time.unique() for i in range(7)]).unique())

In [20]:
#mean
result = tmp.copy()

result["computed_0"] = result.apply(lookup2, axis=1)

computed_tmp_c = result["computed_0"].copy()
computed_tmp_l = result["computed_0"].copy()
computed_tmp_r = result["computed_0"].copy()
for i in range(1,max(smooth_cnt)+1):
    computed_tmp_c = computed_tmp_c.rolling(window=3, center=True).mean().fillna(computed_tmp_c)
    computed_tmp_l = computed_tmp_l.rolling(window=3).mean().fillna(computed_tmp_l)
    computed_tmp_r = computed_tmp_r.iloc[::-1].rolling(window=3).mean().fillna(computed_tmp_r.iloc[::-1]).iloc[::-1]
    if(i in smooth_cnt):
        result["computed_c" + str(i)] = computed_tmp_c.copy()
        result["computed_l" + str(i)] = computed_tmp_l.copy()
        result["computed_r" + str(i)] = computed_tmp_r.copy()
result["computed4"] = result.apply(lambda x: x["computed_" + (str(optimal_smooth[x.dweek][x.time][2]) if (optimal_smooth[x.dweek][x.time][0]!=0) else "") + str(optimal_smooth[x.dweek][x.time][0])] , axis=1)

In [21]:
#train_2018
mean_squared_error(result[result.speed.notna()].speed, result[result.speed.notna()].computed4) 

9.395514104239702

In [24]:
submission = test.merge(result[["datetime","computed4"]], left_on="date",right_on="datetime", how="left")

In [25]:
submission.drop(columns=["date","datetime"], inplace=True)

In [29]:
submission.rename(columns={"computed4":"speed"}, inplace=True)

In [30]:
submission.to_csv("20340468.csv", index=False)